## Data Clean up
In this file we clean up the dataset (Phishing.csv), and write it to a new csv (Phishing_Cleaned.csv)

### Imports

In [1]:
from enum import unique

from IPython import InteractiveShell
from numpy.distutils.conv_template import header

InteractiveShell.ast_node_interactivity = "all"
import pandas as pd 
import numpy as np
from numpy.ma.core import shape


C:\Users\jaert\AppData\Local\Temp\ipykernel_26392\1874912767.py:4: DeprecationWarning: 

  `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
  of the deprecation of `distutils` itself. It will be removed for
  Python >= 3.12. For older Python versions it will remain present.
  It is recommended to use `setuptools < 60.0` for those Python versions.
  For more details, see:
    https://numpy.org/devdocs/reference/distutils_status_migration.html 


  from numpy.distutils.conv_template import header


### Reading the csv
Here we setup pandas to read the csv properly.

In [2]:
phishing_data = pd.read_csv("../data/Phishing.csv", header=0, decimal='.')
phishing_data.head()


,url,source,label,url_length,starts_with_ip,url_entropy,has_punycode,digit_letter_ratio,dot_count,at_count,dash_count,tld_count,domain_has_digits,subdomain_count,nan_char_entropy,has_internal_links,whois_data,domain_age_days
0,apaceast.cloudguest.central.arubanetworks.com,Cisco-Umbrella,legitimate,45,False,3.924535,False,0.0,4,0,0,0,False,3,0.310387,False,"{'domain_name': ['ARUBANETWORKS.COM', 'arubane...",8250.0
1,quintadonoval.com,Majestic,legitimate,17,False,3.572469,False,0.0,1,0,0,0,False,0,0.240439,False,"{'domain_name': ['QUINTADONOVAL.COM', 'quintad...",10106.0
2,nomadfactory.com,Majestic,legitimate,16,False,3.327820,False,0.0,1,0,0,0,False,0,0.250000,False,"{'domain_name': ['NOMADFACTORY.COM', 'nomadfac...",8111.0
3,tvarenasport.com,Majestic,legitimate,16,False,3.500000,False,0.0,1,0,0,0,False,0,0.250000,False,"{'domain_name': ['TVARENASPORT.COM', 'tvarenas...",5542.0
4,widget.cluster.groovehq.com,Cisco-Umbrella,legitimate,27,False,3.930270,False,0.0,3,0,0,0,False,2,0.352214,False,"{'domain_name': 'GROOVEHQ.COM', 'registrar': '...",5098.0


In [3]:
# Checking for null values
"Amount of null values found per column in the dataset"
NullOverview = phishing_data.isnull().sum()
NullOverview


'Amount of null values found per column in the dataset'

url                        1
source                     0
label                      0
url_length                 0
starts_with_ip             0
url_entropy                0
has_punycode               0
digit_letter_ratio         1
dot_count                  0
at_count                   0
dash_count                 0
tld_count                  0
domain_has_digits          0
subdomain_count            0
nan_char_entropy           0
has_internal_links         0
whois_data            545300
domain_age_days       750689
dtype: int64

## Clearing the NaN 
Here we clean the NaN values out of the dataset using different techniques.

In [4]:
"Before removing empty URL"
phishing_data[phishing_data['url'].isnull()]
data_clean = phishing_data.dropna(subset=['url'])

"After cleaning empty URL"
data_clean[data_clean['url'].isnull()]

'Before removing empty URL'

,url,source,label,url_length,starts_with_ip,url_entropy,has_punycode,digit_letter_ratio,dot_count,at_count,dash_count,tld_count,domain_has_digits,subdomain_count,nan_char_entropy,has_internal_links,whois_data,domain_age_days
2026700,NaN,Phishing.Database,phishing,9639,False,-0.0,False,NaN,0,0,0,0,False,0,0.0,False,NaN,NaN


'After cleaning empty URL'

,url,source,label,url_length,starts_with_ip,url_entropy,has_punycode,digit_letter_ratio,dot_count,at_count,dash_count,tld_count,domain_has_digits,subdomain_count,nan_char_entropy,has_internal_links,whois_data,domain_age_days


In [5]:
lowest_number = data_clean['domain_age_days'].min()
highest_number = data_clean['domain_age_days'].max()

print("Lowest domain_age_days in dataset: " + str(lowest_number))
print("Highest domain_age_days in dataset: " + str(highest_number))

nan_count = data_clean['domain_age_days'].isna().sum()
print("\nNumber of NaN values in domain_age_days column:", nan_count)

Lowest domain_age_days in dataset: -86.0
Highest domain_age_days in dataset: 45541.0

Number of NaN values in domain_age_days column: 750688


In this codeblock we change all the negative domain_age_days values to be a NaN, that way we can use imputation to make the data usable

In [6]:
data_clean.loc[data_clean['domain_age_days'] < 0, 'domain_age_days'] = np.nan


In [7]:
lowest_number = data_clean['domain_age_days'].min()
highest_number = data_clean['domain_age_days'].max()

print("Lowest domain_age_days in dataset: " + str(lowest_number))
print("Highest domain_age_days in dataset: " + str(highest_number))

nan_count = data_clean['domain_age_days'].isna().sum()
print("\nNumber of NaN values in domain_age_days column:", nan_count)

Lowest domain_age_days in dataset: 0.0
Highest domain_age_days in dataset: 45541.0

Number of NaN values in domain_age_days column: 750698


Now it's time to Impute the NaN values so there are none remaining, here we use the median because it's less sensitive to outliars 

In [8]:
median_value = data_clean['domain_age_days'].median()

data_clean.loc[data_clean['domain_age_days'].isna(), 'domain_age_days'] = median_value

nan_count_after = data_clean['domain_age_days'].isna().sum()
print("\nNumber of NaN values in domain_age_days column after imputation:", nan_count_after)


Number of NaN values in domain_age_days column after imputation: 0


## Removal of whois_data
Here we remove whois_data because it's hard to use it properly in our model.

In [9]:
data_clean.pop('whois_data')

0          {'domain_name': ['ARUBANETWORKS.COM', 'arubane...
1          {'domain_name': ['QUINTADONOVAL.COM', 'quintad...
2          {'domain_name': ['NOMADFACTORY.COM', 'nomadfac...
3          {'domain_name': ['TVARENASPORT.COM', 'tvarenas...
4          {'domain_name': 'GROOVEHQ.COM', 'registrar': '...
                                 ...                        
2499995    {'domain_name': 'novoatend', 'registrar': None...
2499996                                                  NaN
2499997                                                  NaN
2499998    {'domain_name': 'alternateangle.in', 'registra...
2499999    {'domain_name': None, 'registrar': None, 'whoi...
Name: whois_data, Length: 2499999, dtype: object

## Removal of sources
We remove the sources because all the phishing links came from the same 4 sources, this produces bias in the dataset 

In [10]:
data_clean[(data_clean['source'] != 'Phishing.Database') & (data_clean['label'] == 'phishing') & (data_clean['source'] != 'PhishTank') & (data_clean['source'] != 'OpenPhish-Community')]


,url,source,label,url_length,starts_with_ip,url_entropy,has_punycode,digit_letter_ratio,dot_count,at_count,dash_count,tld_count,domain_has_digits,subdomain_count,nan_char_entropy,has_internal_links,domain_age_days


In [11]:
data_clean.pop('source')

0             Cisco-Umbrella
1                   Majestic
2                   Majestic
3                   Majestic
4             Cisco-Umbrella
                 ...        
2499995    Phishing.Database
2499996    Phishing.Database
2499997    Phishing.Database
2499998    Phishing.Database
2499999    Phishing.Database
Name: source, Length: 2499999, dtype: object

In [12]:
data_clean[(data_clean['url'].str.contains('-randomstring1'))]


,url,label,url_length,starts_with_ip,url_entropy,has_punycode,digit_letter_ratio,dot_count,at_count,dash_count,tld_count,domain_has_digits,subdomain_count,nan_char_entropy,has_internal_links,domain_age_days
1382278,https://npaiaajvaxrjmmhagh.com∕[-randomstring1...,phishing,147,False,4.676039,True,0.035714,5,1,8,2,False,1,0.737305,True,384.0
1560982,https://ztftbcjtlvpsejgcqi.com∕[-randomstring1...,phishing,147,False,4.916021,True,0.035714,5,1,8,2,False,1,0.737305,True,384.0
1664672,https://tzdxqliipbsbyprdbikhwbzgi.com∕[-random...,phishing,161,False,4.939577,True,0.031746,5,1,8,2,False,1,0.689495,True,384.0
1913540,https://lfjifcdhdkrpwhqbuuyigwh.com∕[-randomst...,phishing,157,False,5.010871,True,0.032787,5,1,8,2,False,1,0.702438,True,384.0
2283936,https://obxwijgahveotwmzmnxm.com∕[-randomstrin...,phishing,151,False,4.862656,True,0.034483,5,1,8,2,False,1,0.722904,True,384.0
2309359,https://1wbzjkhyay9f76ev4qoki4r7∕1wbzjkhyay9f7...,phishing,233,False,4.957926,True,0.135802,1,1,21,0,False,0,0.617381,False,353.0
2349192,https://bnv2nzdppjqsnxzo∕bnv2nzdppjqsnxzo∕[-ra...,phishing,216,False,4.692096,True,0.063694,1,1,21,0,False,0,0.653321,False,353.0


In [13]:
data_clean = data_clean[~data_clean['url'].str.contains('-randomstring1')]


In [14]:
data_clean.loc[:, 'label'] = data_clean['label'].astype('category')

print(data_clean['label'])

0          legitimate
1          legitimate
2          legitimate
3          legitimate
4          legitimate
              ...    
2499995      phishing
2499996      phishing
2499997      phishing
2499998      phishing
2499999      phishing
Name: label, Length: 2499992, dtype: object


In [18]:
data_clean.sample(frac=1).to_csv('../data/clean_data.csv', index=False)

## Checks after export

In [20]:
data = pd.read_csv("../data/clean_data.csv", sep=",", header=0)
data.head()

,url,label,url_length,starts_with_ip,url_entropy,has_punycode,digit_letter_ratio,dot_count,at_count,dash_count,tld_count,domain_has_digits,subdomain_count,nan_char_entropy,has_internal_links,domain_age_days
0,https://hotelcasadapraca.com/components/Agrico...,phishing,76,False,4.145551,False,0.0,2,0,0,0,False,0,0.509494,False,4281.0
1,https://area.clientes-ingdiirect.es-eu.org/login,phishing,48,False,4.050614,False,0.0,3,0,2,0,False,2,0.616353,False,4281.0
2,etrxbip.cn,legitimate,10,False,3.321928,False,0.0,1,0,0,0,False,0,0.332193,False,353.0
3,saltiegirl.com,legitimate,14,False,3.521641,False,0.0,1,0,0,0,False,0,0.271954,False,3713.0
4,https://sdndvsdvqwddsdvsdv.page.link/zbFk?drpk...,phishing,50,False,4.326370,False,0.0,2,0,0,0,False,1,0.655042,False,2768.0


## Checking for NaN values

In [21]:
"Amount of null values found per column in the dataset"
NullOverview = data.isnull().sum()
NullOverview

'Amount of null values found per column in the dataset'

url                   0
label                 0
url_length            0
starts_with_ip        0
url_entropy           0
has_punycode          0
digit_letter_ratio    0
dot_count             0
at_count              0
dash_count            0
tld_count             0
domain_has_digits     0
subdomain_count       0
nan_char_entropy      0
has_internal_links    0
domain_age_days       0
dtype: int64